In [1]:
import pandas as pd
import os
from google import genai
from docx import Document
import re

In [2]:
with open('C:/cred_google_gemini.txt', 'r') as file:
    llave = file.read()

In [3]:
def limpiar_texto(x):
    if isinstance(x, str):
        x = re.sub(r"A\\n", "", x)     # eliminar "A\n"
        x = re.sub(r"\\n", " ", x)     # reemplazar "\n" por espacio
        x = re.sub(r"\\t", " ", x)     # reemplazar "\t" por espacio
        x = re.sub(r"•", "", x)        # eliminar viñetas
        x = re.sub(r"\s+", " ", x)  
        #Eliminar "-"
        x= re.sub(r"-", "", x)
        x= re.sub(r"●", "", x)
        return x.strip()
    return x

In [4]:
##Leer matriz de consolidación
matriz=pd.read_excel("H:/Mi unidad/cuestionarios/visiones_dependencias_pgd.xlsx")
matriz.columns=matriz.columns.str.lower()  
#Renombrar segunda columna como "otro"
matriz.rename(columns={matriz.columns[1]: 'otro',
                           matriz.columns[2]: 'corto_plazo_1',
    matriz.columns[3]: 'corto_plazo_2',
    matriz.columns[4]: 'mediano_plazo',
    matriz.columns[5]: 'largo_plazo'
    
    }, inplace=True)
matriz['dependencia'] = matriz.apply(
    lambda row: row["otro"]
    if row['dependencia'] == 'Otro' and pd.notna(row["otro"])
    else row['dependencia'],
    axis=1
)
matriz["dependencia"] = matriz["dependencia"].str.title()


#Eliminar columna otro
del matriz['otro']

#Eliminar espacios en blanco en todas las columnas
matriz = matriz.map(limpiar_texto)


matriz

,dependencia,corto_plazo_1,corto_plazo_2,mediano_plazo,largo_plazo,fuente
0,Secretaría De La Contraloría General De La Cdmx,Lograr retomar la confianza de los ciudadanos ...,Un gobierno justo y transparente con Servidore...,Cumplir con mejorar la ciudad y los servicios ...,Un serviciopúbñico comprometido con el servici...,Mesa 1
1,Secretaría De Cultura,"Generar actividades de convivencia en pueblos,...",Espacio Público y libre acceso a la vida cultu...,Espacio Público y libre acceso a la vida cultu...,Convertir a la Ciudad de México en el escenari...,Mesa 1
2,Secretaría De La Contraloría General De La Cdmx,Consolidar una institución que contribuya al c...,"Consolidar mecanismos de vigilancia, denuncia,...",Implementar herramientas tecnológicas que agil...,Establecer una metodología unificada de fiscal...,Mesa 1
3,Secretaría De Atención Y Participación Ciudadana,Aplicación de los primeros diagnósticos integr...,Consolidación del Diagnóstico Integral de las ...,Disminución de la desigualdad territorial y la...,La Ciudad de México consolidada como una ciuda...,Mesa 1
4,Procuraduría Ambiental Y Del Ordenamiento Terr...,Mediante la atención de las denuncias ciudadan...,Consolidación de una normatividad ambiental y ...,El entendimiento de los beneficios del suelo d...,Aumentar la capacidad de atención de problemát...,Mesa 1
5,Secretaría De Gestión Integral Del Agua,A Conocer el estado actual de infiltración y r...,Implementar acciones estructurales y no estruc...,A Alcanzar la sustentabilidad hídrica del acuí...,Deben haberse construido todas las obras reque...,Mesa 1
6,Secretaría De Las Mujeres,Conformación de 500 redes comunitarias de muje...,Redes comunitarias funcionales acompañadas de ...,Cambios estructurales en la participación ciud...,Red de mujeres reconocida como actor clave en ...,Mesa 1
7,Secretaría De Las Mujeres,Policía de proximidad. Unidades especializadas...,Reducción significativa de la impunidad en del...,Garantía del derecho de las mujeres a vivir li...,Erradicación de los delitos contra la libertad...,Mesa 1
8,Secretaría De Las Mujeres,"Facilitar y agilizar los procesos de denuncia,...",Reducción de cifras de impunidad en los delito...,Acceso a la justicia con perspectiva de género...,Contar con un modelo de justicia feminista que...,Mesa 1
9,Secretaría De Las Mujeres,Estrategia para garantizar el derecho de las m...,a) Se consolida el marco normativo derivado de...,La Defensoría de las Mujeres debe garantizar u...,La Defensoría de las Mujeres debe garantizar u...,Mesa 1


In [5]:
matriz["dependencia"].unique()

array(['Secretaría De La Contraloría General De La Cdmx',
       'Secretaría De Cultura',
       'Secretaría De Atención Y Participación Ciudadana',
       'Procuraduría Ambiental Y Del Ordenamiento Territorial De La Ciudad De México',
       'Secretaría De Gestión Integral Del Agua',
       'Secretaría De Las Mujeres',
       'Secretaría De Trabajo Y Fomento Al Empleo',
       'Instituto De La Juventud',
       'Secretaría De Pueblos Y Barrios Originarios Y Comunidades Indígenas Residentes',
       'Secretaría De Bienestar E Igualdad Social',
       'Consejo Para Prevenir Y Eliminar La Discriminación De La Ciudad De México',
       'Secretaría De Vivienda', 'Secretaría De Desarrollo Económico',
       'Secretaría De Turismo',
       'Fideicomiso Centro Histórico De La Ciudad De México',
       'Secretaría De Salud De La Ciudad De México'], dtype=object)

In [6]:
#Pasar a long
matriz_long = pd.melt(matriz, id_vars=['dependencia'], 
                      value_vars=['corto_plazo_1', 'corto_plazo_2', 'mediano_plazo', 'largo_plazo'],
                      var_name='plazo', value_name='vision')
#Renombrar corto_plazo_1 y corto_plazo_2 como corto_plazo
matriz_long['plazo'] = matriz_long['plazo'].replace({'corto_plazo_1': 'corto_plazo',
                                                    'corto_plazo_2': 'corto_plazo'})
matriz_long = matriz_long.groupby(['dependencia', 'plazo'], as_index=False).agg({
    'vision': lambda x: '. '.join(x).strip()
})
matriz_long

,dependencia,plazo,vision
0,Consejo Para Prevenir Y Eliminar La Discrimina...,corto_plazo,"Para el año 2027, es indispensable haber actua..."
1,Consejo Para Prevenir Y Eliminar La Discrimina...,largo_plazo,"En el horizonte de 2050, la Ciudad de México d..."
2,Consejo Para Prevenir Y Eliminar La Discrimina...,mediano_plazo,"Para 2040, el objetivo es que se haya logrado ..."
3,Fideicomiso Centro Histórico De La Ciudad De M...,corto_plazo,a. Diseñar un Plan Maestro de regeneración int...
4,Fideicomiso Centro Histórico De La Ciudad De M...,largo_plazo,Sumar las acciones que se realicen en el Centr...
5,Fideicomiso Centro Histórico De La Ciudad De M...,mediano_plazo,Fortalecer la colaboración interinstitucional ...
6,Instituto De La Juventud,corto_plazo,"Para 2027, el INJUVE debe consolidar las bases..."
7,Instituto De La Juventud,largo_plazo,"Para 2050, la Ciudad de México deberá garantiz..."
8,Instituto De La Juventud,mediano_plazo,"Para 2040, deben haberse transformado las cond..."
9,Procuraduría Ambiental Y Del Ordenamiento Terr...,corto_plazo,Mediante la atención de las denuncias ciudadan...


In [7]:
# 2) Función que arma el “documento” por grupo
def documento_por_grupo(g, salto="\n\n"):
    corto = salto.join(
        f"Corto plazo: {t.strip()}"
        for t in g.loc[g.plazo.str.contains("largo_plazo", na=False), "vision"]
    )
    mediano  = salto.join(
        f"Objetivo: {t.strip()}"
        for t in g.loc[g.plazo.str.contains("largo_plazo", na=False), "vision"]
    )
    largo  = salto.join(
        f"Estrategia: {t.strip()}"
        for t in g.loc[g.plazo.str.contains("largo_plazo", na=False), "vision"]
    )
    return corto, mediano, largo

In [8]:
# 4. Crear carpeta de salida si no existe
if not os.path.exists("visiones_dependencias"):
    os.makedirs("visiones_dependencias")

In [9]:
grupos = (
    matriz_long.groupby(["dependencia"], dropna=False)
          .apply(documento_por_grupo)
)

print(f"Se generarán {len(grupos)} documentos Word")
grupos

Se generarán 16 documentos Word


C:\Users\claud\AppData\Local\Temp\ipykernel_1892\416768345.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(documento_por_grupo)


dependencia
Consejo Para Prevenir Y Eliminar La Discriminación De La Ciudad De México         (Corto plazo: En el horizonte de 2050, la Ciud...
Fideicomiso Centro Histórico De La Ciudad De México                               (Corto plazo: Sumar las acciones que se realic...
Instituto De La Juventud                                                          (Corto plazo: Para 2050, la Ciudad de México d...
Procuraduría Ambiental Y Del Ordenamiento Territorial De La Ciudad De México      (Corto plazo: Aumentar la capacidad de atenció...
Secretaría De Atención Y Participación Ciudadana                                  (Corto plazo: La Ciudad de México consolidada ...
Secretaría De Bienestar E Igualdad Social                                         (Corto plazo: Garantizar el pleno ejercicio de...
Secretaría De Cultura                                                             (Corto plazo: Convertir a la Ciudad de México ...
Secretaría De Desarrollo Económico                              

In [10]:
client=genai.Client(api_key=llave)

In [20]:
for clave, (corto, mediano, largo) in grupos.items():
    dependencia = clave

    # --- Prompt con prefijos ya incluidos -----------------------------------
    prompt = (
        "Redacta un documento sintético de corrido que genere la visión de la dependencia utilizando la siguiente información:\n\n"
        f"{corto}\n\n{mediano}\n\n{largo}"
        "Dale coherencia. No repitas la información, no uses viñetas ni listas y no inventes información.\n"
        "El documento debe señalar a corto plazo (2030), mediano plazo (2040) y largo plazo (2050).\n"
        "El documento debe ser de no más de una cuartilla.\n"
    )

    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    resumen = resp.candidates[0].content.parts[0].text.strip()


    # --- Crear DOCX ----------------------------------------------------------
    doc = Document()
    doc.add_heading(f"{dependencia}", level=1)
    #doc.add_paragraph(f"Eje: {eje}")
    #doc.add_paragraph(f"Documento fuente: {documento}")
    doc.add_heading("Documento", level=2)
    doc.add_paragraph(resumen)

    # Guardar
    nombre = f"{dependencia[:30]}.docx".replace(" ", "_")
    #nombre = f"{linea[:30]}_{tema[:20]}.docx".replace(" ", "_")
    ruta = os.path.join("visiones_dependencias", nombre)
    doc.save(ruta)
    print("✓ Guardado:", ruta)

✓ Guardado: visiones_dependencias\Consejo_Para_Prevenir_Y_Elimin.docx
✓ Guardado: visiones_dependencias\Fideicomiso_Centro_Histórico_D.docx
✓ Guardado: visiones_dependencias\Instituto_De_La_Juventud.docx
✓ Guardado: visiones_dependencias\Procuraduría_Ambiental_Y_Del_O.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Atención_Y_Parti.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Bienestar_E_Igua.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Cultura.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Desarrollo_Econó.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Gestión_Integral.docx
✓ Guardado: visiones_dependencias\Secretaría_De_La_Contraloría_G.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Las_Mujeres.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Pueblos_Y_Barrio.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Salud_De_La_Ciud.docx
✓ Guardado: visiones_dependencias\Secretaría_De_Trabajo_Y_Foment.docx
✓ Guardado: visiones_dependencias\Secret